# Loading

In [1]:
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import openai
import pandas as pd
import bamboolib as bam
import json
import sqlalchemy
from sqlalchemy import create_engine
import sys
sys.path.append('..')
sys.path.append('../src')

In [3]:
from llm_api import LLM_API

from apikey import apikey
os.environ['OPENAI_API_KEY']=apikey
openai.api_key  = os.getenv('OPENAI_API_KEY')

# Example
 - https://innerjoin.bit.io/make-chatgpt-stop-chatting-and-start-writing-sql-fd5560049ae4


In [4]:

system_prompt = "you are a text-to-SQL translator. You write sqlite code based on plain-language prompts.Do not have line breaks in the output."
user_prompt = """
 - Language sqlite
 - Table = "penguins", columns = [species text, island text, bill_length_mm double precision, bill_depth_mm double precision, flipper_length_mm bigint, body_mass_g bigint, sex text, year bigint]
You are a SQL code translator. Your role is to translate natural language to sqlite. Your only output should be SQL code. Do not include any other text. Only SQL code.
Translate "How many penguins are there in each island and only display islands with more than 10 penguins ?" to a syntactically-correct sqlite query. 
"""

# list of dicts specifying roles and content
query = [{"role":"system", "content": system_prompt},
         {"role":"user", "content": user_prompt}]

print(f'{system_prompt}')
print(f'{user_prompt}')

you are a text-to-SQL translator. You write sqlite code based on plain-language prompts.Do not have line breaks in the output.

 - Language sqlite
 - Table = "penguins", columns = [species text, island text, bill_length_mm double precision, bill_depth_mm double precision, flipper_length_mm bigint, body_mass_g bigint, sex text, year bigint]
You are a SQL code translator. Your role is to translate natural language to sqlite. Your only output should be SQL code. Do not include any other text. Only SQL code.
Translate "How many penguins are there in each island and only display islands with more than 10 penguins ?" to a syntactically-correct sqlite query. 



In [5]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=query,
)
print(completion.choices[0].message)

{
  "content": "SELECT island, COUNT(*) AS total_penguins \nFROM penguins \nGROUP BY island \nHAVING total_penguins > 10;",
  "role": "assistant"
}


In [30]:
check_sql_syntax(completion.choices[0].message.content)

SQL syntax is valid.


In [9]:
print(user_prompt)


 - Language PostgreSQL
 - Table = "penguins", columns = [species text, island text, bill_length_mm double precision, bill_depth_mm double precision, flipper_length_mm bigint, body_mass_g bigint, sex text, year bigint]
You are a SQL code translator. Your role is to translate natural language to PostgreSQL. Your only output should be SQL code. Do not include any other text. Only SQL code.
Translate "How many penguins are there in each island?" to a syntactically-correct PostgreSQL query.



In [22]:
import sqlparse

def check_sql_syntax(sql_string):
    try:
        parsed = sqlparse.parse(sql_string)
        print("SQL syntax is valid.")
    except sqlparse.exceptions.SQLParseError as e:
        print("Invalid SQL syntax:", str(e))

# Example usage
sql_string = "SELECT * FROM my_table WHERE column = 'value'"
check_sql_syntax(sql_string)

SQL syntax is valid.


In [35]:
print(sqlparse.format(completion.choices[0].message.content, reindent=True, keyword_case='upper'))

SELECT island,
       COUNT(*) AS num_penguins
FROM penguins
GROUP BY island
HAVING num_penguins > 10;


# Example 
- https://dev.classmethod.jp/articles/convert-a-natural-language-query-into-an-sql-query-using-openai-api/

In [9]:
df = pd.read_csv("../data/fruits_raw.csv")

In [11]:
# df

In [12]:
temp_db = sqlalchemy.create_engine('sqlite:///:memory:')

In [170]:
def pushDb(df,tbl="FRUITS"):
    '''TODO : Purge all existing tables before pushing'''
    df.to_sql(name=tbl, con=temp_db, if_exists='replace')    

In [83]:
def execDB(sqlString):
    with temp_db.connect() as conn:
        result = conn.execute(sqlalchemy.text(sqlString))
    return pd.DataFrame(result.all()) 

In [138]:
def getColsDB():
    ''' Get Col names from table (assume single table)'''
    retString=None
    inspector = sqlalchemy.inspect(temp_db)
    table_names = inspector.get_table_names()
    table_name=table_names[0]
    retString=f"Table = {table_name}, "
    columns = inspector.get_columns(table_name)
    retString+=f"Columns = ["
    for col in columns:
        retString+=f"{col['name']} {col['type']}, "
    retString+="]"
    return retString

In [144]:
def nlp_query():
    nlp_text= input("Enter the query: ")
    return nlp_text

In [185]:
def get_sql(nlp_query='Can you list the number of fruits per shape ?'):
    system_prompt = "you are a text-to-SQL translator. You write sqlite code based on plain-language prompts.Do not have line breaks in the output."
    prompt=f"""
     - Language = sqlite
     - {getColsDB()}
    You are a SQL code translator. Your role is to translate natural language text delimited by triple backticks to sqlite. Your only output should be SQL code. Do not include any other text. Only SQL code.
    Translate ```{nlp_query}``` to a syntactically-correct sqlite query. 
    """
    print(prompt)
    response=LLM_API.get_completion(prompt,system_prompt)
    print(f'response:\t{response}')
    return response

In [186]:
tt=get_sql()


     - Language = sqlite
     - Table = FRUITS, Columns = [index BIGINT, Fruit TEXT, Shape TEXT, Taste TEXT, Season TEXT, Calories BIGINT, Fat FLOAT, Carbohydrates BIGINT, Sugar BIGINT, Protein FLOAT, Fiber FLOAT, ]
    You are a SQL code translator. Your role is to translate natural language text delimited by triple backticks to sqlite. Your only output should be SQL code. Do not include any other text. Only SQL code.
    Translate ```Can you list the number of fruits per shape ?``` to a syntactically-correct sqlite query. 
    
messages:
[{'role': 'system', 'content': 'you are a text-to-SQL translator. You write sqlite code based on plain-language prompts.Do not have line breaks in the output.'}, {'role': 'user', 'content': '\n     - Language = sqlite\n     - Table = FRUITS, Columns = [index BIGINT, Fruit TEXT, Shape TEXT, Taste TEXT, Season TEXT, Calories BIGINT, Fat FLOAT, Carbohydrates BIGINT, Sugar BIGINT, Protein FLOAT, Fiber FLOAT, ]\n    You are a SQL code translator. Your ro

In [187]:
execDB(tt)

         Shape  Num_Fruits
0         Long           2
1       Oblong           2
2         Oval           9
3        Round          27
4        Stalk           1
5  Star-shaped           1

In [164]:
execDB(completion.choices[0]['message']['content'])

         Shape  Number_of_Fruits
0         Long                 2
1       Oblong                 2
2         Oval                 9
3        Round                27
4        Stalk                 1
5  Star-shaped                 1

In [168]:
completion['usage']['total_tokens']

182

In [76]:
# temp_db = sqlalchemy.create_engine('sqlite:///:memory:')
# df.to_sql(name="FRUITS", con=temp_db, if_exists='replace')
# temp_db.dispose()
# with temp_db.connect() as conn:
# result = conn.execute(sqlalchemy.text())

# results_output=result.all()
# pd.DataFrame(results_output)
# # Iterate over the table names
# for table_name in table_names:
#     # Get the column information for each table
    
    
#     # Print the table name
#     print("Table:", table_name)
    
#     # Print the column names and types
#     for column in columns:
#         print("Column:", column['name'])
#         print("Type:", column['type'])
        
#     print()

In [173]:
# system_prompt = "you are a text-to-SQL translator. You write sqlite code based on plain-language prompts.Do not have line breaks in the output."
# user_prompt=f"""
#  - Language = sqlite
#  - {getColsDB()}
# You are a SQL code translator. Your role is to translate natural language text delimited by triple backticks to sqlite. Your only output should be SQL code. Do not include any other text. Only SQL code.
# Translate ```{nlp_query()}``` to a syntactically-correct sqlite query. 
# """

# # list of dicts specifying roles and content
# query = [{"role":"system", "content": system_prompt},
#          {"role":"user", "content": user_prompt}]

# print(f'{system_prompt}')
# print(f'{user_prompt}')


# completion = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=query,
# )
# print(completion.choices[0].message)

In [175]:
# def printColsDB():
#     ''' Get Col names from table (assume single table)'''
#     inspector = sqlalchemy.inspect(temp_db)
#     table_names = inspector.get_table_names()
#     table_name=table_names[0]
#     print(f"Table = {table_name}")
#     columns = inspector.get_columns(table_name)
#     print(f"Columns = ",end='[')
#     for col in columns:
#         print(f"{col['name']} {col['type']},",end=' ')
#     print("]")